<a href="https://colab.research.google.com/github/HuyenNguyenHelen/LING-5412/blob/main/Assignment2_LogisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing libraries that will be used 
import numpy as np
import tarfile
import glob
import re
import pandas as pd
#from nltk.corpus import stopwords
import nltk
nltk.download('wordnet')
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Loading the data

In [2]:
# Untar the dataset
my_tar = tarfile.open('/content/review_polarity.tar.gz')
my_tar.extractall('/content/') 
my_tar.close()


In [3]:
# Exploring the data sizes

paths_pos = glob.glob('/content/txt_sentoken/pos/*.txt')
paths_neg = glob.glob('/content/txt_sentoken/neg/*.txt')
pos_neg_paths = paths_pos + paths_neg

n_pos = len(paths_pos)
n_neg = len(paths_neg)

print('the number of positive instances: {} \nthe number of positive instances: {}'.format(n_pos, n_neg))

the number of positive instances: 1000 
the number of positive instances: 1000


# Exploring the data

In [4]:
# Exploring the words in the dataset

from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords = set(stopwords.words('english'))

def tokenizer (doc):
  #doc = doc.lower() # Lowercase documents
  return re.split("\W+", doc)   # return a list of tokens without punctuations

# def BOW (doc):
#   bow = set()
#   for token in tokenizer (doc):
#     bow.add(token)
#   return list(bow)

#def word_counter (doc):

def stopword_remover (bow):
  filtered_bow = [w for w in bow if not w.lower() in stopwords]
  return filtered_bow

def top_freq_w (freq_dic, top_n, stopword_removing = ''):
  sorted_dic = {k:v for k, v in sorted(freq_dic.items(), key = lambda item: item[1], reverse=True)}
  if stopword_removing is False:
    return {k:v for k, v in list(sorted_dic.items())[:top_n]}
  elif stopword_removing is True:
    filtered_dic = {k: v for k, v in sorted_dic.items() if k not in stopwords}
    return {k:v for k, v in list(filtered_dic.items())[:top_n]}
  


word_freq = {}
for path in pos_neg_paths:
  fo = open(path)
  doc = fo.read()
  for token in tokenizer (doc):
    word_freq[token] = word_freq.get(token,0)+1

top_100_w = top_freq_w(word_freq, 100, stopword_removing = False) 

print('the number of unique words in the dataset: ', len(word_freq.keys()))
print ('top 100 most frequent words:\n', top_100_w )
print('\nthe number of words in the top 100 which are stopwords: ', len([w for w in top_100_w.keys() if w in stopwords]))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
the number of unique words in the dataset:  39697
top 100 most frequent words:
 {'the': 76529, 'a': 38106, 'and': 35576, 'of': 34123, 'to': 31937, 'is': 25195, 'in': 21822, 's': 18513, 'it': 16107, 'that': 15924, 'as': 11378, 'with': 10792, 'for': 9961, 'his': 9587, 'this': 9578, 'film': 9517, 'i': 8889, 'he': 8864, 'but': 8634, 'on': 7385, 'are': 6949, 't': 6410, 'by': 6261, 'be': 6174, 'one': 5852, 'movie': 5771, 'an': 5744, 'who': 5692, 'not': 5577, 'you': 5316, 'from': 4999, 'at': 4986, 'was': 4940, 'have': 4901, 'they': 4825, 'has': 4719, 'her': 4522, 'all': 4373, 'there': 3770, 'like': 3690, 'so': 3683, 'out': 3637, 'about': 3523, 'up': 3405, 'more': 3347, 'what': 3322, 'when': 3258, 'which': 3161, 'or': 3148, 'she': 3141, 'their': 3122, 'some': 2985, 'just': 2905, 'can': 2882, 'if': 2799, 'we': 2775, 'him': 2633, 'into': 2623, 'even': 2565, 'only': 2495, 'than':

In [5]:
# Reformating the dataset into csv for convenience 
def to_df (folder):
  data_dic = {}
  data_dic['doc'], data_dic['label'] = [], []
  for file in folder:
    fo = open(file)
    doc = fo.read()
    data_dic['doc'].append(doc)
    if 'pos' in file:
      data_dic['label'].append(1)
    elif 'neg' in file:
      data_dic['label'].append(0)
    else:
      print('error', file)
  df = pd.DataFrame.from_dict(data_dic)
  return df
    
data = to_df(pos_neg_paths)

data.head()

,doc,label
0,"ingredients : london gal , fate , true love , ...",1
1,"quiz show , an almost perfectly accurate true ...",1
2,after a stylistic detour with mrs . \nparker a...,1
3,"all great things come to an end , and the dot-...",1
4,melvin udall is a heartless man . \nhe spends ...,1


# Data preprocessing

In [6]:
# Data preprocessing
stemmer = nltk.stem.porter.PorterStemmer()
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
def preprocessor (text):
  ## removing punctuations and characters
  text = re.sub(r'[^\w\s]', '', text)
  # stripping
  text = ' '.join([w.strip() for w in text.split()])
  # print(text)
  ## lowcasing
  text = text.lower()
  # ## removing stopword
  text = stopword_remover (text.split())
  # ##stemmming
  text = [stemmer.stem(w) for w in text]
  # ## lematization
  text = [lemmatizer.lemmatize(w) for w in text]
  return ' '.join([w for w in text])

data['doc'] = data['doc'].apply(lambda x:  preprocessor (x) )
data 

,doc,label
0,ingredi london gal fate true love run joke mon...,1
1,quiz show almost perfectli accur true stori ba...,1
2,stylist detour mr parker viciou circl despit u...,1
3,great thing come end dotcom era embodi perfect...,1
4,melvin udal heartless man spend day insid spac...,1
...,...,...
1995,often similar littl boy lost park right ventur...,0
1996,13th warrior reek badli melodrama poor act car...,0
1997,accord hitchcock variou filmmak isol motel din...,0
1998,warn follow review contain spoiler cast gari s...,0


# Developing a Logistic Regression Model

In [7]:
# Preparing vocabulary
## As required, we will use 1000 most frequent word, excluding stopwords
### Preprocessing data for building vocabulary
cleaned_word_freq = {}
for path in pos_neg_paths:
  fo = open(path)
  doc = fo.read()
  cleaned_doc = preprocessor(doc)
  for token in tokenizer (cleaned_doc):
    cleaned_word_freq[token] = cleaned_word_freq.get(token,0)+1

vocabulary = top_freq_w(cleaned_word_freq, 1000, stopword_removing = True) 


In [8]:
## remove this code
# Feature engineering
# def feature_extractor (doc):
#   doc_vec = []
#   for feature in vocabulary.keys():
#     feature_count = 0
#     if feature in tokenizer (doc):
#       feature_count+=1
#     else:
#       feature_count+=0
#     doc_vec.append(feature_count) 
#   return doc_vec

In [9]:


# def feature_extractor (doc):
#   doc_vec = []
#   token_list = tokenizer (doc)
#   for feature in vocabulary.keys():
#     # feature_count=0
#     feature_count = token_list.count(feature)
#     doc_vec.append(feature_count) 
#   return doc_vec

# X = data['doc'].apply(lambda x: feature_extractor(x))
# y = data['label']


def feature_extractor (doc):
  doc_vec = []
  token_list = tokenizer (doc)
  for feature in vocabulary.keys():
    if feature in token_list:
      doc_vec.append(1)
    else:
       doc_vec.append(0)
  return doc_vec

X = data['doc'].apply(lambda x: feature_extractor(x))
y = data['label']

In [10]:

X = X.apply(pd.Series)
X.columns = vocabulary.keys()
X

,film,movi,one,like,charact,get,make,time,scene,even,good,play,stori,see,would,much,also,go,way,seem,look,end,two,take,first,come,well,work,thing,year,realli,plot,know,perform,littl,life,peopl,love,could,bad,...,rush,realist,scare,manner,command,standard,menac,spent,adam,agre,cinematographi,front,ground,budget,fairli,pair,virtual,suddenli,fantasi,connect,disturb,90,appropri,godzilla,brown,grant,cultur,greatest,store,trip,key,fascin,cute,brief,cameo,count,foot,addit,satir,bug
0,1,0,1,1,1,1,0,0,1,0,0,1,1,0,0,0,1,1,1,1,1,0,1,1,0,0,0,1,0,1,1,0,1,1,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,1,1,0,0,1,0,1,0,1,1,0,1,1,1,0,1,1,0,1,1,0,1,0,0,1,1,0,1,0,0,0,0,1,0,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,1,0,1,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,1,1,0,0,0,1,0,0,0,1,1,1,1,0,1,1,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,1,0,1,0,0,1,1,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,1,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1,1,1,0,1,0,1,1,1,0,0,1,0,0,1,0,1,0,0,1,0,1,1,1,1,0,0,1,1,1,0,1,1,1,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1996,1,1,1,1,1,0,1,1,1,1,0,0,0,0,1,0,0,0,0,1,1,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1997,1,0,1,0,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1998,1,1,1,1,1,1,1,1,1,1,0,0,0,1,1,0,1,1,1,1,0,1,1,1,1,1,0,1,0,0,0,1,1,1,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0


In [11]:
# Spliting the dataset for training and testing
X_train, X_test, y_train, y_test = train_test_split (X[vocabulary.keys()], y , train_size = 0.8, random_state = 42, shuffle = True, stratify=data['label'])
print ('Shapes of X_train, y_train: ', X_train.shape, y_train.shape)
print ('Shapes of X_test, y_test: ', X_test.shape, y_test.shape)

Shapes of X_train, y_train:  (1600, 1000) (1600,)
Shapes of X_test, y_test:  (400, 1000) (400,)


In [12]:
X_train.values

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])

In [26]:
# class LogisticRegression ():
#   def __init__ (lr = '', n_iter = 10):



def predict(X, weight, bias):
    z = np.dot(weight, X.values.T) + bias
    y_pred = sigmoid(z)
    y_class = [1 if i > 0.5 else 0 for i in y_pred]
    return y_class

def sigmoid (z):
  p=1/(1+np.exp(-z))
  return p

# Printing model performance 
def printing_eval_scores (y_true, y_pred, report=''):
  print('accuracy score: {}'.format(sklearn.metrics.accuracy_score(y_true, y_pred)))
  print('precision score: {}'.format(sklearn.metrics.precision_score(y_true, y_pred)))
  print('recall score: {}'.format(sklearn.metrics.recall_score(y_true, y_pred)))
  print('F1 score: {}'.format(sklearn.metrics.f1_score(y_true, y_pred)))
  if report is True:
    print(classification_report(y_true, y_pred))

def computing_gradient (X, Y, weight, bias):
  for x,y in zip (X.values, Y):
    z = np.dot(weight, x) + bias
    y_pred = sigmoid(z)
    #loss = -(y*log(y_pred)+(1-y)*log(1-y_pred))
    d_weight = np.dot((y_pred - y), x)
    d_bias = (y_pred - y)
    weight -= lr*d_weight
    bias -= lr*d_bias
  return weight, bias

lr = 0.1
n_iter = 10
# weight = None
# bias = None
weight = np.zeros(X.shape[1])
bias = 0
for iter in range(n_iter):
  print('\n====================batch %s=======================' % iter)
  # for x,y in zip (X_train.values,y_train):
  #   z = np.dot(weight, x) + bias
  #   y_pred = sigmoid(z)
  #   #loss = -(y*log(y_pred)+(1-y)*log(1-y_pred))
  #   d_weight = np.dot((y_pred - y), x)
  #   d_bias = (y_pred - y)
  #   weight -= lr*d_weight
  #   bias -= lr*d_bias
  weight, bias = computing_gradient (X_train, y_train, weight, bias)
  y_predict = predict(X_test,weight, bias)
  # Model performing
  ## on training set
  print('Model performance on training set:')
  printing_eval_scores (y_train, predict(X_train,weight, bias), report = False)

  ## on test set
  print('\nModel performance on test set:')
  printing_eval_scores (y_test, y_predict, report = True)


    



====================batch 0=======================
Model performance on training set:
accuracy score: 0.8775
precision score: 0.9428152492668622
recall score: 0.80375
F1 score: 0.8677462887989204

Model performance on test set:
accuracy score: 0.795
precision score: 0.8554216867469879
recall score: 0.71
F1 score: 0.7759562841530054
              precision    recall  f1-score   support

           0       0.75      0.88      0.81       200
           1       0.86      0.71      0.78       200

    accuracy                           0.80       400
   macro avg       0.80      0.79      0.79       400
weighted avg       0.80      0.80      0.79       400


====================batch 1=======================
Model performance on training set:
accuracy score: 0.92
precision score: 0.9653739612188366
recall score: 0.87125
F1 score: 0.9159001314060446

Model performance on test set:
accuracy score: 0.8
precision score: 0.8448275862068966
recall score: 0.735
F1 score: 0.786096256684492
       

#Minibatch training

In [30]:


# class LogisticRegression ():
#   def __init__ (lr = '', n_iter = 10):

def computing_MiniBatch_gradient (X, Y, weight, bias, batch_size):
  n_instances, n_features = X.shape
  i=0 
  while i< round(n_instances/batch_size):
    m = batch_size * i
    n = m + batch_size
    sum_w = 0
    sum_b = 0
    for x,y in zip (X[m:n].values,Y[m:n]):
      z = np.dot(weight, x) + bias
      y_pred = sigmoid(z)
      #loss = -(y*log(y_pred)+(1-y)*log(1-y_pred))
      sum_w += np.dot((y_pred - y), x)
      sum_b += (y_pred - y)
    d_weight = (1/batch_size)*sum_w
    d_bias = (1/batch_size)*sum_b
    weight -= lr*d_weight
    bias -= lr*d_bias
    i+=1
  return  weight, bias

lr = 0.1
n_iter = 10
weight = np.zeros(X.shape[1])
bias = 0
for iter in range(n_iter):
  print('\n====================batch %s=======================' % iter)
  weight, bias = computing_MiniBatch_gradient (X_train, y_train, weight, bias,  batch_size = 32)
  y_predict = predict(X_test,weight, bias)
  
  # Model performing
  ## on training set
  print('Model performance on training set:')
  printing_eval_scores (y_train, predict(X_train,weight, bias), report = False)

  ## on test set
  print('\nModel performance on test set:')
  printing_eval_scores (y_test, y_predict, report = True)

    


====================batch 0=======================
Model performance on training set:
accuracy score: 0.838125
precision score: 0.7826541274817137
recall score: 0.93625
F1 score: 0.8525896414342629

Model performance on test set:
accuracy score: 0.84
precision score: 0.788135593220339
recall score: 0.93
F1 score: 0.8532110091743119
              precision    recall  f1-score   support

           0       0.91      0.75      0.82       200
           1       0.79      0.93      0.85       200

    accuracy                           0.84       400
   macro avg       0.85      0.84      0.84       400
weighted avg       0.85      0.84      0.84       400


====================batch 1=======================
Model performance on training set:
accuracy score: 0.873125
precision score: 0.8312985571587126
recall score: 0.93625
F1 score: 0.8806584362139918

Model performance on test set:
accuracy score: 0.845
precision score: 0.8108108108108109
recall score: 0.9
F1 score: 0.8530805687203792
  

# L2 Regularization

In [42]:


# class LogisticRegression ():
#   def __init__ (lr = '', n_iter = 10):

def MiniBatch_gradient_L2 (X, Y, weight, bias,alpha, batch_size):
  n_instances, n_features = X.shape
  i=0 
  while i< round(n_instances/batch_size):
    m = batch_size * i
    n = m + batch_size
    sum_w = 0
    sum_b = 0
    for x,y in zip (X[m:n].values,Y[m:n]):
      z = np.dot(weight, x) + bias
      y_pred = sigmoid(z)
      w_i = np.dot((y_pred - y), x)
      b_i = y_pred - y
      sum_w +=  w_i  + (alpha * weight) # L2
      sum_b += b_i
    d_weight = (1/batch_size)*sum_w 
    d_bias = (1/batch_size)*sum_b
    weight -= lr*d_weight
    bias -= lr*d_bias
    i+=1
  return  weight, bias

lr = 0.1
n_iter = 10
alpha = 0.01
weight = np.zeros(X.shape[1])
bias = 0
for iter in range(n_iter):
  print('\n====================batch %s=======================' % iter)
  weight, bias = MiniBatch_gradient_L2 (X_train, y_train, alpha = alpha,  weight=weight, bias=bias, batch_size = 32)
  y_predict = predict(X_test,weight, bias)

  # Model performing
  ## on training set
  print('Model performance on training set:')
  printing_eval_scores (y_train, predict(X_train,weight, bias), report = False)

  ## on test set
  print('\nModel performance on test set:')
  printing_eval_scores (y_test, y_predict, report = True)


    


====================batch 0=======================
Model performance on training set:
accuracy score: 0.836875
precision score: 0.781021897810219
recall score: 0.93625
F1 score: 0.8516202387720296

Model performance on test set:
accuracy score: 0.84
precision score: 0.788135593220339
recall score: 0.93
F1 score: 0.8532110091743119
              precision    recall  f1-score   support

           0       0.91      0.75      0.82       200
           1       0.79      0.93      0.85       200

    accuracy                           0.84       400
   macro avg       0.85      0.84      0.84       400
weighted avg       0.85      0.84      0.84       400


====================batch 1=======================
Model performance on training set:
accuracy score: 0.871875
precision score: 0.8294573643410853
recall score: 0.93625
F1 score: 0.8796241926012919

Model performance on test set:
accuracy score: 0.8475
precision score: 0.8116591928251121
recall score: 0.905
F1 score: 0.8557919621749409


In [16]:
feature_weights = {}
for i in range(len(weight)):
  feature_weights[list(vocabulary.keys())[i]] = weight[i]

sorted_feature_weights = {k:v for k, v in sorted(feature_weights.items(), key = lambda item: item[1], reverse=True)}

# Print the weights learned for each class
print('The most important features of POSITIVE class: ')
for k, v in list(sorted_feature_weights.items())[:100]:
  print ('{}: {:.5f}'. format(k,v))

print('\n==============================================')
print('The most important features of NEGATIVE class: ')
for k, v in list(sorted_feature_weights.items())[:-100:-1]: 
  print ('{}: {:.5f}'. format(k,v))

The most important features of POSITIVE class: 
sometim: 0.43061
great: 0.38293
hilari: 0.37989
memor: 0.36271
perform: 0.33587
excel: 0.33369
flaw: 0.33169
perfect: 0.32485
also: 0.32103
job: 0.31929
overal: 0.31293
definit: 0.30809
fun: 0.29313
equal: 0.29114
thank: 0.28403
especi: 0.27571
enjoy: 0.27373
seen: 0.27150
portray: 0.26862
american: 0.26837
perfectli: 0.26366
true: 0.26285
day: 0.26114
ive: 0.25985
deserv: 0.25361
side: 0.25273
fiction: 0.24805
life: 0.24665
brilliant: 0.24135
solid: 0.23994
best: 0.23715
share: 0.23545
differ: 0.23352
le: 0.23341
normal: 0.23141
well: 0.23117
today: 0.22643
david: 0.22265
yet: 0.22232
visual: 0.22196
period: 0.21835
realist: 0.21709
support: 0.21256
surpris: 0.21253
entertain: 0.21232
throughout: 0.21105
togeth: 0.21099
oscar: 0.20809
delight: 0.20595
nice: 0.20379
world: 0.20127
sever: 0.20004
see: 0.19681
summer: 0.19663
follow: 0.19639
wonder: 0.19598
peopl: 0.19528
take: 0.19373
allow: 0.19124
histori: 0.19124
extrem: 0.19061
truli: 

In [43]:
# Sklearn Logistic Regression Model
from sklearn.linear_model import LogisticRegression
sk_lr = LogisticRegression(solver='lbfgs', max_iter=150).fit(X_train, y_train )
y_predict = sk_lr.predict(X_test)

# Model performing
## on training set
print('Model performance on training set:')
printing_eval_scores (y_train, sk_lr.predict(X_train))

## on test set
print('\n===========================')
print('Model performance on test set:')
printing_eval_scores (y_test, y_predict)

Model performance on training set:
accuracy score: 1.0
precision score: 1.0
recall score: 1.0
F1 score: 1.0

Model performance on test set:
accuracy score: 0.8275
precision score: 0.8291457286432161
recall score: 0.825
F1 score: 0.8270676691729322
